# A Snowpark Workflow for Linear Mixed Effects

Using a car auction dataset we will determine the effect of mileage on price 

## You will learn in Snowflake/Snowpark Python
- how to train and score a model using statsmodel (comparable to R LME4)
- how to register that model


In [ ]:
#import necessary packages --- needs statmodels and snowflake-ml-python 1.5.0 (select it in the upper right packages list)
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

import statsmodels.api as sm
import statsmodels.formula.api as smf


## Note: Comparing R lmer to statsmodels MixedLM
The statsmodels implementation of linear mixed models (MixedLM) closely follows the approach outlined in Lindstrom and Bates (JASA 1988). This is also the approach followed in the R package LME4. Other packages such as Stata, SAS, etc. should also be consistent with this approach, as the basic techniques in this area are mostly mature.

[This site](https://www.statsmodels.org/stable/examples/notebooks/generated/mixed_lm_example.html) shows how linear mixed models can be fit using the MixedLM procedure in statsmodels with results from R (LME4) included for comparison.


In [ ]:
#snowflake libs
from snowflake.ml.registry import Registry
from snowflake.snowpark.session import Session
from snowflake.snowpark.context import get_active_session
session = get_active_session()

source_table_name = "LAB_DATA.PUBLIC.CARS_5000"
clean_table_name = "LAB_DATA.PUBLIC.CARS_CLEANED_ALL_COLUMNS"

MY_NAME='MY_NAME' ##!!!CHANGE THIS TO YOUR NAME!!!
session.sql("create schema if not exists "+MY_NAME+";").collect()
session.use_schema(MY_NAME)

print(session.get_current_database())
print(session.get_current_schema())
print(session.get_current_role())

In [ ]:
#familiar python code running on Scalable Snowflake architecture and leveraging a bit of Snowpark to collect data
pd_df_raw = session.table(source_table_name).to_pandas()
pd_df_cleaned = session.table(clean_table_name).to_pandas()
pd_df_cleaned.describe()

In [ ]:
plt.hist(pd_df_raw['PRICE'], bins=20, edgecolor='grey')
# Rotate x labels
plt.xticks(rotation=45)
# Show the plot
plt.tight_layout() 
# Add titles and labels
plt.title('Histogram of Price')
plt.xlabel('Price')
plt.ylabel('Frequency')

# Show the plot
plt.show()

In [ ]:
plt.hist(pd_df_raw['MILEAGE'], bins=20, edgecolor='grey')
# Rotate x labels
plt.xticks(rotation=45)
# Show the plot
plt.tight_layout() 
# Add titles and labels
plt.title('Histogram of Mileage')
plt.xlabel('MILEAGE')
plt.ylabel('Frequency')

# Show the plot
plt.show()

In [ ]:
scaler = MinMaxScaler()
# Fit and transform the mileage column
pd_df_raw['MILEAGE_SCALED'] = scaler.fit_transform(pd_df_raw[['MILEAGE']])



In [ ]:
#split training data out
def get_training_columns(columns):
    clean_columns_train = []
    for col in columns:
        if col != 'PRICE':
                clean_columns_train.append(col)
    return clean_columns_train
    
get_training_columns = get_training_columns(pd_df_raw)

X_train, X_test, y_train, y_test = train_test_split(pd_df_raw[get_training_columns], pd_df_raw['PRICE'],test_size=0.2)


In [ ]:
pd_df_raw.columns

In [ ]:
# Define the formula for the mixed effects model
formula = 'PRICE ~ MILEAGE + C(MODEL_NAME)'

# Fit the mixed linear model
# Here, we assume 'brand' as the grouping variable for random effects
model = smf.mixedlm(formula, pd_df_raw, groups=pd_df_raw['MAKE'])
result = model.fit()

# Display the model summary
print(result.summary())

In [ ]:
# and let's plot the predictions
performance = pd.DataFrame()
performance["residuals"] = result.resid.values
performance["mileage_scaled"] = pd_df_raw["MILEAGE_SCALED"]
performance["predicted"] = result.fittedvalues


In [ ]:
# Plot predicted values against scaled mileage
plt.figure(figsize=(10, 6))

# Plot predicted vs. mileage_scaled
plt.scatter(performance["mileage_scaled"], performance["predicted"], color='blue', label='Predicted')
plt.xlabel('Scaled Mileage')
plt.ylabel('Predicted Price')
plt.title('Predicted Price vs. Scaled Mileage')
plt.legend()

# Show plot
plt.show()

# Optional: Plot residuals
plt.figure(figsize=(10, 6))
plt.scatter(performance["mileage_scaled"], performance["residuals"], color='red', label='Residuals')
plt.xlabel('Scaled Mileage')
plt.ylabel('Residuals')
plt.title('Residuals vs. Scaled Mileage')
plt.axhline(y=0, color='black', linestyle='--', linewidth=1)
plt.legend()

# Show plot
plt.show()


[Registering 'Custom' Models](https://docs.snowflake.com/en/developer-guide/snowpark-ml/model-registry/custom-models)

The Snowflake model registry allows you to register (log) models and use them for inference inside Snowflake. The registry supports several types of models:

- Snowpark ML Modeling
- scikit-learn
- XGBoost
- LightGBM
- CatBoost
- PyTorch
- TensorFlow
- MLFlow PyFunc
- Sentence Transformer
- Hugging Face pipeline

The model registry API also allows you to log other types of models, including those trained using external tools or obtained from open source repositories, as long as they are serializable and derived from the snowflake.ml.model.custom_model.CustomModel class.

See also this [blog post](https://medium.com/snowflake/integrating-machine-learning-models-with-snowpark-ml-a-guide-for-azureml-and-sagemaker-users-735292843a7b) for information about importing models from AWS SageMaker or from Azure ML.